In [ ]:
import matplotlib.pylab as plt

import numpy as np
import pandas as pd
# import holoviews as hv
# hv.notebook_extension()
%matplotlib inline
# %config InlineBackend.figure_format = 'svg'

golden_mean = (np.sqrt(5) - 1) / 2 # Aesthetic ratio
fig_width_pt = 246.0 # Columnwidth
inches_per_pt = 1.0 / 72.27 # Convert pt to inches
fig_width = fig_width_pt * inches_per_pt
fig_height = fig_width * golden_mean # height in inches
fig_size = [fig_width, fig_height]

params = {'backend': 'ps',
          'axes.labelsize': 10,
          'font.size': 10,
          'legend.fontsize': 8,
          'xtick.labelsize': 8,
          'ytick.labelsize': 8,
          'text.usetex': True,
          'figure.figsize': fig_size,
          'font.family': 'serif',
          'font.serif': 'Computer Modern Roman',
          'legend.frameon': True,
          'savefig.dpi': 300
         }

plt.rcParams.update(params)
plt.rc('text.latex', preamble=[r'\usepackage{color}', r'\usepackage{bm}'])

import matplotlib.colors as colors
    
class HistogramNormalize(colors.Normalize):
    def __init__(self, data, vmin=None, vmax=None, mixing_degree=1):
        self.mixing_degree = mixing_degree
        if vmin is not None:
            data = data[data >= vmin]
        if vmax is not None:
            data = data[data <= vmax]

        self.sorted_data = np.sort(data.flatten())
        colors.Normalize.__init__(self, vmin, vmax)

    def __call__(self, value, clip=None):
        hist_norm = np.ma.masked_array(np.searchsorted(self.sorted_data, value) /
                                       len(self.sorted_data))
        linear_norm = super().__call__(value, clip)
        return self.mixing_degree * hist_norm + (1 - self.mixing_degree) * linear_norm

    
def open_mtx(fname):
    with open(fname, 'rb') as f:
        line1 = f.readline()
        *shape, entry_size = [int(i) for i in  f.readline().split()]
        print('shape: ', shape)
        data = np.fromfile(f, (np.float64 if entry_size == 8 else np.float32))
        data = data.reshape(shape).squeeze()
    
    keys = ['units', 'plot_settings', 'xname', 'xmin', 'xmax', 
            'yname', 'ymin', 'ymax', 'zname', 'zmin', 'zmax']
    values = line1.decode("utf-8").replace('\n', '').split(',')
    meta_data = dict(zip(keys, values))
    meta_data = {}
    for key, val in zip(keys, values):
        try:
            meta_data[key] = float(val)
        except:
            meta_data[key] = val
    return meta_data, data

# Fig. 4 disorder plots

In [ ]:
from scipy.signal import argrelextrema
df = pd.read_hdf('data/I_c(B_x)_mu10,50meV_disorder0,10,20,30,40,50,60,70,80,90,100,110meV_combinations_of_effects.hdf')
gb = df.groupby(['orbital', 'g', 'alpha', 'mu', 'disorder', 'salt'])
total = gb.get_group((True, 50, 20, 50, 0, 0))

In [ ]:
df['disorder'].unique()

In [ ]:
fig, axs = plt.subplots(4, 1, sharey=False, sharex=True, figsize=(fig_width, 1.5*fig_height), 
                        gridspec_kw={'height_ratios':[2, 1, 2, 1]})

plt.subplots_adjust(bottom=0.2, left=0.125, right=0.90, top=0.9, hspace=0.25, wspace=0.15)
current = lambda orbital=True, g=50, alpha=20, mu=10, disorder=0: (
    gb.get_group((orbital, g, alpha, mu, disorder, 0))['current_c'].values * 2.09757)
# multiplied by 2e/hbar * k_B * T
# http://www.wolframalpha.com/input/?i=(2+*+electronic+charge+%2F+hbar+)+*+boltzmann+constant+*+50e-3+kelvin

phase = lambda orbital=True, g=50, alpha=20, mu=10, disorder=0: gb.get_group((orbital, g, alpha, mu, disorder, 0))['phase_c'].values
B_x = total['B_x']

def plot_curves(ax, mu, plot=current):
    args = {'lw': 0.6, 'alpha': 1}
    if mu == 10:
        disorder=70
    else:
        disorder=100
    ax.plot(B_x, plot(disorder=disorder, mu=mu), label='all effects', c='b', **args)
    ax.plot(B_x, plot(disorder=disorder, orbital=False, mu=mu), label=r'$\bm{A}=0$', c='g', **args)
    ax.plot(B_x, plot(disorder=disorder, g=0, mu=mu)*1.025, label='$g=0$', c='r', **args) # small off-set to make the curves not overlap
    ax.plot(B_x, plot(disorder=disorder, alpha=0, mu=mu), label=r'$\alpha=0$', c='m', **args)
    ax.plot(B_x, plot(disorder=disorder, orbital=False, alpha=0, mu=mu), label=r'$\alpha=0, \; \bm{A}=0$', c='c', **args)
    ax.plot(B_x, plot(disorder=disorder, alpha=0, g=0, mu=mu), label=r'$\alpha=0, \; g=0$', c='y', **args)
    ax.set_xlim(0, 1)
    return ax

ymax = 0.1
axs[0] = plot_curves(axs[0], 10)
# axs[0].set_yticks([0, 0.2, 0.4, 0.6])
axs[0].set_ylim(0, ymax)
axs[0].set_xlabel('')
axs[0].set_ylabel('$I_c$ [nA]')
leg0 = axs[0].legend(loc='upper center', bbox_to_anchor=(0.5, 1.9), ncol=2, prop={'size': 6})

axs[0].text(0.01, 0.8*ymax, '(a)')

ymax = 10
axs[2] = plot_curves(axs[2], 50)
# axs[2].set_yticks([0, 20, 40, 60, 80])
axs[2].set_ylim(0, ymax)
axs[2].set_ylabel('$I_c$ [nA]')
axs[2].text(0.01, 0.8*ymax, '(c)')

for i in argrelextrema(current(orbital=False, alpha=0, mu=10, disorder=70), np.less):
    x = B_x.iloc[i]
    y1, y2 = 0, ymax
    axs[0].plot((x, x), (y1, y2), 'c--')

for i in argrelextrema(current(orbital=False, alpha=0, mu=50, disorder=110), np.less):
    x = B_x.iloc[i]
    y1, y2 = 0, ymax
    axs[2].plot((x, x), (y1, y2), 'c--')
    
def first_min_max(arr):
    return B_x.iloc[argrelextrema(arr, np.less)[0]], B_x.iloc[argrelextrema(arr, np.greater)[0]]

B1, B2 = first_min_max(current(orbital=False, alpha=0, mu=10)) # for next cell
B3, B4 = first_min_max(current(orbital=False, alpha=0, mu=50)) # for next cell

axs[1] = plot_curves(axs[1], 10, phase)
axs[1].set_yticks([-3.14, 0, 3.14])
axs[1].set_yticklabels(['$-\pi$', 0, '$\pi$'])
axs[1].set_ylabel(r'$\theta_c$')
axs[1].text(0.01, -2.15, '(b)')

axs[3] = plot_curves(axs[3], 50, phase)
axs[3].set_yticks([-3.14, 0, 3.14])
axs[3].set_yticklabels(['$-\pi$', 0, '$\pi$'])
axs[3].set_xlabel('$B_x$ [$T$]')
axs[3].set_ylabel(r'$\theta_c$')
axs[3].text(0.01, -2.1, '(d)')
fig.savefig('figures/critical_currents_disorder.pdf', format='pdf', dpi=300, bbox_inches="tight", additional_artists=[leg0])
plt.show()

# Current as function of $\mu$ and $B_x$

In [ ]:
df = pd.read_hdf('data/I_c(B_x,_mu)_combinations_of_effects.hdf')
gb = df.groupby(['orbital', 'g', 'alpha'])

gr = gb.get_group((True, 50, 20))
for (orbital, g, alpha), gr in gb:
    fig = plt.figure(figsize=(fig_width, fig_height))
    ax = fig.add_axes([0.1, 0.2, 0.71, 0.85])
    extent = (gr.B_x.min(), gr.B_x.max(), gr.mu.min(), gr.mu.max())
    norm = HistogramNormalize(gr.current_c.values)
    kwargs = dict(extent=extent, cmap='viridis', aspect='auto', vmax=20, 
                  norm=norm, origin='lower')
    im = ax.imshow(gr.pivot_table(index='B_x', columns='mu', values='current_c').T, **kwargs)
    ax.set_xlabel('Magnetic field $B_x$ [$T$]')
    ax.set_ylabel(r'$\mu$ [meV]')
    ax.set_title(r'$\bm{{A}} {}0, \; g={}, \; \alpha={}$'.format((r'\ne' if orbital else '='), g, alpha))
    cax = fig.add_axes([0.83, 0.2, 0.04, 0.85])
    cbar_ticks = [0, 5, 10, 15, 20, 25]
    cb = fig.colorbar(im, cax=cax, extend='max', label=r'$I_c$ [meV$/\Phi_0$]', ticks=cbar_ticks)
    cb.ax.set_yticklabels(cbar_ticks)
    plt.show()